In [21]:
import sys
import numpy as np
import torch
from torch import tensor
import torchvision 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
from matplotlib import pyplot as plt
import json
from collections import namedtuple

sequential_path = '/Users/ebrahimfeghhi/NCEL/rnn_modeling/activity_analysis/'
sys.path.insert(1, sequential_path)
from compute_SI import *

neuro_jit_path = '/Users/ebrahimfeghhi/NCEL/rnn_modeling/models/'
sys.path.insert(1, neuro_jit_path)
from Neuro_RNN_JIT import NeuroRNNCell, NeuroRNNLayer, NeuroRNN

data_path = '/Users/ebrahimfeghhi/NCEL/rnn_modeling/models/'
sys.path.insert(1, neuro_jit_path)
from Neuro_RNN_JIT import NeuroRNNCell, NeuroRNNLayer, NeuroRNN

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
model_name = 'det_trial'
base_file = 'saved_models/' + model_name + '/' 
mi_file = base_file + 'model_info_dict.json'                                   

with open(mi_file, 'r') as f:
    model_info_dict = json.load(f)

In [36]:
batch_size = 1
num_trials=20
seq_len=30
dataset_folder_path = '/Users/ebrahimfeghhi/NCEL/rnn_modeling/datasets/'
sys.path.insert(1, dataset_folder_path)
from retrieve_datasets import retrieve
test_set = retrieve(data_code=0, num_trials=20, n_in=model_info_dict['input_size'])
testLoader = DataLoader(test_set, batch_size=batch_size,
        shuffle=True)

Input shape:  (20, 30, 10)


In [37]:
RNNState = namedtuple('RNNState', ['r', 'I'])
state = RNNState(torch.zeros(model_info_dict['hidden_size'], batch_size), torch.zeros(model_info_dict['hidden_size'], batch_size))
rnnLayer = NeuroRNNLayer(NeuroRNNCell, model_info_dict['input_size'], model_info_dict['hidden_size'], model_info_dict['alpha_r'], model_info_dict['alpha_s'], model_info_dict['nonlin'])
model = NeuroRNN(rnnLayer, model_info_dict['output_dim'], ratio=.8)
model.eval()

NeuroRNN(
  (rnn_layer): NeuroRNNLayer(
    (cell): NeuroRNNCell()
  )
)

In [38]:
total_hidden_state = torch.zeros(num_trials, seq_len, model_info_dict['hidden_size'])
for i, (data, targets) in enumerate(testLoader):
    output, hidden_state = model(data, state)
    total_hidden_state[i] = hidden_state.permute(0,2,1).detach()
hidr = total_hidden_state.numpy()